In [19]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import warnings

In [20]:
warnings.filterwarnings('ignore')

In [21]:
data = pd.read_csv('tmdb_5000_movies.csv')
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [22]:
columns = data.columns.to_list()

In [23]:
good_list = columns[:1] + columns[8:9] + columns[11:14] + columns[-3:]

In [24]:
new_data = data[good_list]

In [25]:
new_data.head(3)

,budget,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,237000000,150.437577,2009-12-10,2787965087,162.0,Avatar,7.2,11800
1,300000000,139.082615,2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,107.376788,2015-10-26,880674609,148.0,Spectre,6.3,4466


In [26]:
new_data.dtypes

budget            int64
popularity      float64
release_date     object
revenue           int64
runtime         float64
title            object
vote_average    float64
vote_count        int64
dtype: object

In [27]:
new_data.index = new_data.title

In [28]:
new_data.drop('title', axis=1, inplace=True)

In [29]:
new_data.head()

,budget,popularity,release_date,revenue,runtime,vote_average,vote_count
title,,,,,,,
Avatar,237000000,150.437577,2009-12-10,2787965087,162.0,7.2,11800
Pirates of the Caribbean: At World's End,300000000,139.082615,2007-05-19,961000000,169.0,6.9,4500
Spectre,245000000,107.376788,2015-10-26,880674609,148.0,6.3,4466
The Dark Knight Rises,250000000,112.312950,2012-07-16,1084939099,165.0,7.6,9106
John Carter,260000000,43.926995,2012-03-07,284139100,132.0,6.1,2124
